In [1]:
import langchain
print(langchain.__version__)

1.2.0


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
import dotenv
from dotenv import load_dotenv
import os
from langchain_core.tools import tool,Tool
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv()
serp_api_key=os.getenv("SERP_API_KEY")
google_api_key=os.getenv("GOOGLE_API_KEY")

In [5]:
prompt='''You are an expert educator and quiz master for advanced technical students.
Your goal is to create a quiz based strictly on the following text context.
Text Context:   "{text}"

Instructions to be followed:
1. Create {num_questions} multiple-choice questions(MCQs) based only on the Text above"
2. For each question, provide 4 options (A, B, C, D)
3. Identfiy the correct answer
4. Ensure the questions test understanding, not just simple words matching.
5. Difficulty level : {difficulty} (eg. easy,medium or hard)

Output Format: 
You must output a strictly valid JSON string. Do not output markdown code blocks.
The JSON format must be list of objects like this: 
[
    {{
        "question": "The question text here?",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_answer": "Option B",
        "explanation": "A brief explanation of why B is correct."
    }}
]
'''

In [6]:
prompt_template=PromptTemplate(input_variables=["text","num_questions","difficulty"],template=prompt)

In [7]:
prompt=(prompt_template.format(text='''International Debut: He made his ODI debut for India on August 18, 2008, against Sri Lanka.
World Cup Success: Kohli was a key member of the Indian team that won the 2011 ODI World Cup, the 2013 ICC Champions Trophy, and the 2024 T20 World Cup.
Most ODI Centuries: He holds the world record for the most centuries in One Day Internationals (ODIs), surpassing Sachin Tendulkar's long-standing record with his 50th ODI century during the 2023 World Cup.
Record Run Scorer: He is the fastest batsman to reach multiple thousand-run milestones in ODIs, including 8,000, 10,000, 12,000, and 14,000 runs.
Successful Test Captain: As India's most successful Test captain, he led the team to a historic Test series win in Australia in 2018-19 and helped them maintain the #1 Test ranking for a long period.
ICC Awards: His numerous accolades include the Sir Garfield Sobers Trophy (ICC Cricketer of the Year) in 2017 and 2018, and the ICC Men's ODI Cricketer of the Decade (2011–2020).
IPL Career: In the Indian Premier League (IPL), he has played exclusively for the Royal Challengers Bengaluru (RCB) since 2008 and led them to their first title in the 2025 season.
Retirement Announcements: Kohli retired from T20 International cricket in 2024 after India's T20 World Cup victory and announced his retirement from Test cricket in May 2025.
Personal Life: He is married to popular Bollywood actress Anushka Sharma, and they have two children together, a daughter named Vamika and a son named Akaay.
Off-Field Influence: Known for his discipline and passion, he is a role model for aspiring athletes and a highly influential global figure with a significant net worth from endorsements and business ventures.'''
, num_questions=10,difficulty='easy'))

In [8]:
serpapi=SerpAPIWrapper()
search_tool=Tool(name="search", func=serpapi.run, description="Useful for when you need to answer questions about current events")

In [9]:
client=ChatGoogleGenerativeAI(model="gemini-2.0-flash",tools=[search_tool])

Unexpected argument 'tools' provided to ChatGoogleGenerativeAI.
c:\ALL PROJECTS\BrainBolt\venv\Lib\site-packages\IPython\core\interactiveshell.py:3701: UserWarning: WARNING! tools is not default parameter.
                tools was transferred to model_kwargs.
                Please confirm that tools is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:

response = client.invoke(prompt)
print(response.content)